#### 1. Запросить курсы любой валюты за указанный период. Сохранить полученные данные в базу данных Mongo.

In [1]:
from zeep import Client
from pprint import pprint
from pymongo import MongoClient, errors
from decimal import Decimal
import datetime
import math
import numpy as np

In [2]:
def create_db(database_name, index, unique=True):
    try:
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[database_name]
        if index:
            db[database_name].create_index(index, unique=unique)
        return db[database_name]
    except Exception as e:
        return e


def add_to_db(database, document):
    try:
        return database.insert_one(document)
    except errors.DuplicateKeyError as e:
        return e

    
def update_to_db(database, document):
    try:
        return database.update_one(document, upsert=True)
    except Exception as e:
        return e
    
    
def enum_valutes(seld, database_name, index, unique=True):
    try:
        url = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
        client = Client(url)
        result = client.service.EnumValutes(seld)
    
        db = create_db(database_name, index, unique)
    
        for value in result._value_1._value_1:
            doc = {
                'Vcode': value['EnumValutes']['Vcode'],
                'Vname': value['EnumValutes']['Vname'],
                'VEngname': value['EnumValutes']['VEngname'],
                'VcharCode': value['EnumValutes']['VcharCode']
            }
            update_to_db(db, doc)
        return db
    except Exception as e:
        return e
    

def get_currency_code(database, vcharcode):
    try:
        vcode = database.find_one({'VcharCode': vcharcode})['Vcode']
        return vcode
    except Exception as e:
        return e

    
def get_curs_dynamic(database_name, index, currency_database, vcharcode, from_date, to_date):
    try:
        valuta_code = get_currency_code(currency_database, vcharcode)
        
        url = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
        client = Client(url)
        result = client.service.GetCursDynamic(from_date, to_date, valuta_code)
        
        db = create_db(database_name, index)
        
        for value in result._value_1._value_1:
            doc = {
                'VcharCode': vcharcode,
                'CursDate': value['ValuteCursDynamic']['CursDate'],
                'Vcurs': float(value['ValuteCursDynamic']['Vcurs'])
            }
            update_to_db(db, doc)
        return db
    except Exception as e:
        return e

In [3]:
currency_db = enum_valutes(False, 'currency', 'VcharCode', True)

In [4]:
currency_db.find_one({'VcharCode': 'USD'})

{'_id': ObjectId('5d6104d3a7d92ada83d64f2c'),
 'Vcode': 'R01235              ',
 'Vname': 'Доллар США                                                                                                                                                                                                                                                    ',
 'VEngname': 'US Dollar                                                                                                                                                                                                                                                     ',
 'VcharCode': 'USD'}

In [5]:
exchange_rate_db = get_curs_dynamic('exchange_rate', False, currency_db, 'USD', '2018-01-01', '2019-07-31')
exchange_rate_db = get_curs_dynamic('exchange_rate', False, currency_db, 'EUR', '2018-01-01', '2019-07-31')

In [6]:
exchange_rate_db.find_one({'VcharCode': 'USD'})

{'_id': ObjectId('5d610ba0a7d92ada83d656fd'),
 'VcharCode': 'USD',
 'CursDate': datetime.datetime(2018, 1, 9, 21, 0),
 'Vcurs': 57.0463}

In [7]:
exchange_rate_db.find_one({'VcharCode': 'EUR'})

{'_id': ObjectId('5d610ba1a7d92ada83d6587f'),
 'VcharCode': 'EUR',
 'CursDate': datetime.datetime(2018, 1, 9, 21, 0),
 'Vcurs': 68.2103}

#### 2. Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.

In [8]:
def get_max_exchacnge_rate_diff(database, from_date, to_date, vcharcode):
    try:
        s_date = datetime.datetime.strptime(from_date, '%Y-%m-%d')
        e_date = datetime.datetime.strptime(to_date, '%Y-%m-%d')

        objects = database.find({'VcharCode': vcharcode, 'CursDate': {'$gte': s_date, '$lte': e_date}})
        
        min_value, max_value = math.inf, -math.inf
        
        for obj in objects:
            v = obj['Vcurs']
            d = obj['CursDate'].date()
            if v < min_value:
                min_value = v
                min_date = d
            elif v > max_value:
                max_value = v
                max_date = d
                
        return {
                'Min': (min_date.isoformat(), min_value),
                'Max': (max_date.isoformat(), max_value),
                'Diff': np.round(max_value - min_value, 4)
        }
    except Exception as e:
        return e

In [9]:
get_max_exchacnge_rate_diff(exchange_rate_db, '2018-01-01', '2018-12-31', 'USD')

{'Min': ('2018-02-27', 55.6717),
 'Max': ('2018-09-11', 69.9744),
 'Diff': 14.3027}

#### 3. Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>.

In [10]:
def get_max_exchange_rate_diff_pretty(database, from_date, to_date, vcharcode):
    try:
        s_date = datetime.datetime.strptime(from_date, '%Y-%m-%d')
        e_date = datetime.datetime.strptime(to_date, '%Y-%m-%d')

        objects = database.find({'VcharCode': vcharcode, 'CursDate': {'$gte': s_date, '$lte': e_date}})
        
        min_value, max_value = math.inf, -math.inf
        
        for obj in objects:
            v = obj['Vcurs']
            d = obj['CursDate'].date()
            if v < min_value:
                min_value = v
                min_date = d
            elif v > max_value:
                max_value = v
                max_date = d
                
        return f'Валюту {vcharcode} выгодно было купить {min_date.isoformat()} по цене {min_value}, ' \
               f'а продать {max_date.isoformat()} по цене {max_value}. ' \
               f'Прибыль: {np.round(max_value - min_value, 4)}'
    except Exception as e:
        return e

In [11]:
get_max_exchange_rate_diff_pretty(exchange_rate_db, '2018-01-01', '2018-12-31', 'EUR')

'Валюту EUR выгодно было купить 2018-01-10 по цене 67.8841, а продать 2018-09-11 по цене 81.3942. Прибыль: 13.5101'

#### 4. Доработать приложение по вакансиям. При наличии вакансии с зарплатой отличной от руб. приложение должно осущетсвлять автоматический перевод по курсу валюты на сегодняшний день. Информацию получать через SOAP-сервис центробанка.

In [12]:
import pandas as pd
import requests
import json
import re
import hashlib
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [13]:
def get_curse_on_date(date=datetime.date.today().strftime('%Y-%m-%d'), vcharcode='USD'):
    try:
        url = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
        client = Client(url)
        result = client.service.GetCursOnDate(date)
        
        for value in result._value_1._value_1:
            if vcharcode == value['ValuteCursOnDate']['VchCode']:
                return float(value['ValuteCursOnDate']['Vcurs'])
        return float(1)
    except Exception as e:
        return e
    

In [14]:
get_curse_on_date()

65.6046

In [15]:
def get_html(link, headers):
    with requests.session() as s:
        r = s.get(link, headers=headers)
    if r.status_code == 200:
        return r.text
    else:
        return False
    

In [16]:
def get_hh_vacancies(search_queries=[], page_count=1, headers={'User-agent': 'Mozilla/5.0'}):

    try:

        def get_salary_info(link, headers):
            html = get_html(link, headers)
            _min_salary, _max_salary = 0, 0
            _currency = 'РУБ'
            if html:
                soup = bs(html, 'html.parser')
                tags = soup.find_all('p', class_='vacancy-salary')
                _salary = re.findall(r'\d+', re.sub(r'\s', '', bs(str(tags[0])).get_text()))
                _currency_text = re.findall(r'.+\d\s(\w{3}).+', bs(str(tags[0])).get_text().upper())
                if len(_currency_text) > 0 and _currency_text[0] != 'РУБ':
                    _currency = _currency_text[0]
                    _exchange_rate = get_curse_on_date(vcharcode=_currency)
                    _salary = [_exchange_rate * float(_value) for _value in _salary]
                if len(_salary) == 1:
                    _min_salary = _max_salary = _salary[0]
                elif len(_salary) == 2:
                    _min_salary,  _max_salary = _salary
                return _min_salary, _max_salary, _currency
            else:
                return _min_salary, _max_salary, _currency

        def get_vacancy_info(search_queries, page_count, headers):
            if len(search_queries) > 0:
                _df_list = []
                for search_query in search_queries:
                    text = '+'.join(search_query.split(' ')).capitalize()
                    for i in range(page_count):
                        link = f'https://hh.ru/search/vacancy?area=1&text={text}&page={i}'
                        html = get_html(link, headers)
                        if html:
                            soup = bs(html, 'html.parser')
                            tags = soup.find_all('a', class_='bloko-link HH-LinkModifier')
                            _vacancies = {
                                'id': [],
                                'name': [],
                                'min_salary': [],
                                'max_salary': [],
                                'currency': [],
                                'link': []
                            }
                            for tag in tags:
                                _id = 'hh' + re.findall(r'(\d+)\??', tag['href'])[0]
                                # _name = bs(str(tag)).get_text()
                                _name = tag.get_text()
                                _min_salary, _max_salary, _currency = get_salary_info(tag['href'], headers)
                                _link = tag['href']
                                _vacancies['id'].append(hashlib.sha256(_id.encode('utf-8')).hexdigest())
                                _vacancies['name'].append(_name)
                                _vacancies['min_salary'].append(_min_salary)
                                _vacancies['max_salary'].append(_max_salary)
                                _vacancies['currency'].append(_currency)
                                _vacancies['link'].append(_link)
                            _df_temp = pd.DataFrame(_vacancies)
                        else:
                            return False
                        _df_list.append(_df_temp)
                return pd.concat(_df_list).drop_duplicates(subset='id').reset_index(drop=True)
            else:
                return False

        return get_vacancy_info(search_queries, page_count, headers)
    
    except Exception as e:
        
        return e
    

In [17]:
search_queries = ['разработчик']
df = get_hh_vacancies(search_queries, page_count=3)

In [18]:
df[df.currency != 'РУБ']

,id,name,min_salary,max_salary,currency,link
51,216841d18e65b272e6111b7bbb977ac5d9f1b8f67c4aa5...,Middle Android Developer(java/kotlin),164012,203374,USD,https://hh.ru/vacancy/33165996?query=%D0%A0%D0...
